## Algoritmo Random Forest
- Previsão do posicionamento da Abelha Trigona Spinipes no período de 2021-2040, 2041-2060, 2061-80 e 2081-2100

### Configuração de Ambiente 


In [22]:
# Instalar dependências necessárias
# pip install pandas geopandas rasterio scikit-learn matplotlib streamlit folium streamlit-folium

In [23]:
# Importar bibliotecas (adicionando pathlib)
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.features import geometry_mask
import glob
import os
from pathlib import Path  # <<< IMPORTAMOS A BIBLIOTECA PATHLIB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import warnings

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

# --- DEFINIÇÃO AUTOMÁTICA DOS CAMINHOS ---
# Esta linha mágica encontra o caminho da pasta do projeto automaticamente!
# Path.cwd() pega o diretório atual (.../abelhas_extensao/notebooks)
# .parent sobe um nível (.../abelhas_extensao)
BASE_DIR = Path.cwd().parent

# Definir os caminhos completos usando o operador / do pathlib
OCORRENCIAS_PATH = BASE_DIR / 'data' / 'ocorrencias.csv'
CLIMA_ATUAL_PATH = BASE_DIR / 'data' / 'clima_atual'
BRASIL_SHAPE_PATH = BASE_DIR / 'data' / 'BR_UF_2024'
PASTA_PREVISOES = BASE_DIR / 'data' / 'previsoes_futuras'

# Criar a pasta para salvar as previsões, se ela não existir
PASTA_PREVISOES.mkdir(parents=True, exist_ok=True)

print("Ambiente configurado e caminhos definidos automaticamente com pathlib!")
print(f"Pasta raiz do projeto encontrada: {BASE_DIR}")
print(f"Caminho do arquivo de ocorrências: {OCORRENCIAS_PATH}")

Ambiente configurado e caminhos definidos automaticamente com pathlib!
Pasta raiz do projeto encontrada: c:\icev\extensao\abelhas_extensao
Caminho do arquivo de ocorrências: c:\icev\extensao\abelhas_extensao\data\ocorrencias.csv


### Tratamento e Carregamento dos Dados


#### Carregar Dados de Ocorrência e Mapa do Brasil

In [24]:
# Carregar dados de ocorrência de forma robusta (VERSÃO FINAL CORRIGIDA)
try:
    print("Tentando carregar o arquivo 'ocorrencias.csv'...")
    ocorrencias_df_full = pd.read_csv(
        OCORRENCIAS_PATH,
        sep='\t',                     # <<< A MUDANÇA CHAVE! Diz ao pandas para usar tabulação como separador.
        comment='#',
        on_bad_lines='skip',
        low_memory=False,
        encoding='utf-8-sig'
    )
    
    # Agora, selecionamos apenas as colunas que nos interessamos e removemos valores nulos
    ocorrencias_df = ocorrencias_df_full[['decimalLatitude', 'decimalLongitude']].dropna()
    
    print("✅ Arquivo carregado com sucesso!")
    print(f"Total de pontos de ocorrência carregados: {len(ocorrencias_df)}")
    print(ocorrencias_df.head())

except FileNotFoundError:
    print(f"❌ Erro: Arquivo não encontrado em '{OCORRENCIAS_PATH}'. Verifique se o caminho está correto.")
except KeyError:
    print("❌ Erro: As colunas 'decimalLatitude' ou 'decimalLongitude' não foram encontradas no arquivo.")
    print("Verifique os nomes das colunas no cabeçalho do seu CSV.")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado ao carregar o arquivo: {e}")

# Converter o DataFrame para um GeoDataFrame
gdf_ocorrencias = gpd.GeoDataFrame(
    ocorrencias_df,
    geometry=gpd.points_from_xy(ocorrencias_df.decimalLongitude, ocorrencias_df.decimalLatitude),
    crs="EPSG:4326"
)

# Carregar o mapa do Brasil (shapefile)
shapefile_brasil = glob.glob(os.path.join(BRASIL_SHAPE_PATH, "*.shp"))[0]
brasil_gdf = gpd.read_file(shapefile_brasil)

# Unir todos os estados em um único polígono do Brasil
brasil_poligono = brasil_gdf.unary_union

print("\nMapa do Brasil e pontos de ocorrência carregados com sucesso.")

Tentando carregar o arquivo 'ocorrencias.csv'...
✅ Arquivo carregado com sucesso!
Total de pontos de ocorrência carregados: 15244
   decimalLatitude  decimalLongitude
0          -9.2963          -75.9972
1         -23.4671          -56.4886
2         -23.4671          -56.4886
3         -23.4671          -56.4886
4         -23.4671          -56.4886

Mapa do Brasil e pontos de ocorrência carregados com sucesso.


#### Carregar e empilhar Dados Climáticos Atuais 

In [25]:
# Listar todos os arquivos .tif de clima atual, em ordem alfabética
clima_files = sorted(glob.glob(os.path.join(CLIMA_ATUAL_PATH, "*.tif")))

# Abrir o primeiro arquivo para obter metadados
with rasterio.open(clima_files[0]) as src:
    meta = src.meta

# Atualizar os metadados para o novo raster empilhado (agora com 19 bandas)
meta.update(count=len(clima_files))

# Criar o caminho para o arquivo empilhado
stack_path = os.path.join(CLIMA_ATUAL_PATH, "clima_atual_stack.tif")

# Empilhar os rasters em um único arquivo
with rasterio.open(stack_path, 'w', **meta) as dst:
    for i, file in enumerate(clima_files, 1):
        with rasterio.open(file) as src:
            dst.write(src.read(1), i)

print(f"Rasters climáticos atuais empilhados em: {stack_path}")

Rasters climáticos atuais empilhados em: c:\icev\extensao\abelhas_extensao\data\clima_atual\clima_atual_stack.tif


#### Gerar Dados de Pseudo-Ausência
- Geramos pontos de pseudo-ausência em áreas aleatórias, mas longe dos pontos de presença, para treinar o modelo.

In [26]:
# --- VERSÃO MELHORADA PARA GERAR PSEUDO-AUSÊNCIAS ---
from shapely.geometry import Point

# Número de pontos de pseudo-ausência (mantemos o mesmo número)
num_pseudo_ausencias = len(gdf_ocorrencias) * 2

# Isso torna a tarefa de classificação mais realista e desafiadora
pseudo_ausencias_points = []
while len(pseudo_ausencias_points) < num_pseudo_ausencias:
    # Obter limites geográficos do Brasil
    minx, miny, maxx, maxy = brasil_poligono.bounds
    
    # Gerar um ponto aleatório dentro desses limites
    random_point = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
    
    # Verificar se o ponto está dentro do Brasil (sem a verificação de buffer)
    if brasil_poligono.contains(random_point):
        pseudo_ausencias_points.append(random_point)

# Criar um GeoDataFrame para as pseudo-ausências
gdf_pseudo_ausencias = gpd.GeoDataFrame(
    geometry=pseudo_ausencias_points,
    crs="EPSG:4326"
)

print(f"Gerados {len(gdf_pseudo_ausencias)} pontos de pseudo-ausência realistas em todo o Brasil.")

Gerados 30488 pontos de pseudo-ausência realistas em todo o Brasil.


#### Criar um Conjunto de Dados de Treinamento Final

In [27]:
# Função para extrair valores do raster para um GeoDataFrame
def extract_raster_values(gdf, raster_path):
    with rasterio.open(raster_path) as src:
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
        values = [val for val in src.sample(coords)]
    return np.array(values)

# Extrair valores para presenças e pseudo-ausências
valores_presenca = extract_raster_values(gdf_ocorrencias, stack_path)
valores_ausencia = extract_raster_values(gdf_pseudo_ausencias, stack_path)

# Criar o dataset final
X = np.vstack((valores_presenca, valores_ausencia))
y = np.array([1] * len(valores_presenca) + [0] * len(valores_ausencia))

# Nomes das features (bio1 a bio19)
feature_names = [os.path.basename(f).split('.')[0] for f in sorted(clima_files)]

# Criar um DataFrame para visualização
df_treinamento = pd.DataFrame(X, columns=feature_names)
df_treinamento['presenca'] = y

print("Conjunto de dados de treinamento criado:")
print(df_treinamento.head())
print(f"\nShape de X: {X.shape}, Shape de y: {y.shape}")

Conjunto de dados de treinamento criado:
         wc2        wc2        wc2     wc2    wc2   wc2        wc2     wc2  \
0  24.271093  24.590834  24.027500  2825.0  363.0  98.0  42.891392  1032.0   
1  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
2  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
3  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   
4  23.157845  26.684959  19.153584  1513.0  184.0  57.0  34.639950   520.0   

     wc2    wc2     wc2        wc2        wc2         wc2        wc2  \
0  335.0  719.0  1020.0  11.812813  87.883148   28.844713  30.877750   
1  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
2  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
3  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   
4  198.0  474.0   198.0  11.594021  57.734829  317.950592  32.429501   

         wc2        wc2        wc2        wc2  presenca  

### Treinamento do Modelo Random Forest


#### Dividir os Dados e Treinar o Modelo

In [28]:
# --- VERSÃO MELHORADA: TREINAMENTO E AVALIAÇÃO ROBUSTA ---

# Preparar os dados de treino (mesmo código de antes)
valores_presenca = extract_raster_values(gdf_ocorrencias, stack_path)
valores_ausencia = extract_raster_values(gdf_pseudo_ausencias, stack_path)
X = np.vstack((valores_presenca, valores_ausencia))
y = np.array([1] * len(valores_presenca) + [0] * len(valores_ausencia))

# --- Validação Cruzada para uma Avaliação Mais Confiável ---
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Inicializar o modelo com parâmetros para reduzir overfitting
# max_depth: limita a profundidade das árvores
# min_samples_leaf: exige um número mínimo de amostras em uma folha
rf_model = RandomForestClassifier(
    n_estimators=200, 
    random_state=42, 
    n_jobs=-1, 
    class_weight='balanced',
    max_depth=15,          # <<< NOVO: Limita a complexidade
    min_samples_leaf=5     # <<< NOVO: Evita folhas muito específicas
)

print("Avaliando o modelo com Validação Cruzada (5 folds)...")
# StratifiedKFold mantém a proporção de classes em cada fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular a acurácia em cada fold
scores = cross_val_score(rf_model, X, y, cv=cv, scoring='accuracy')

print(f"\nAcurácias em cada fold: {scores}")
print(f"Acurácia Média (CV): {scores.mean():.4f}")
print(f"Desvio Padrão da Acurácia: {scores.std():.4f}")

# Agora, treinamos o modelo final com TODOS os dados disponíveis
print("\nTreinando o modelo final com todos os dados...")
rf_model.fit(X, y)
print("Modelo final treinado!")

# Analisar a importância das variáveis (com o modelo final)
importancias = pd.DataFrame({
    'variavel': feature_names,
    'importancia': rf_model.feature_importances_
}).sort_values('importancia', ascending=False)

print("\nImportância das Variáveis Climáticas (modelo final):")
print(importancias)

Avaliando o modelo com Validação Cruzada (5 folds)...

Acurácias em cada fold: [0.97398054 0.97496447 0.97824185 0.97375902 0.97703914]
Acurácia Média (CV): 0.9756
Desvio Padrão da Acurácia: 0.0018

Treinando o modelo final com todos os dados...
Modelo final treinado!

Importância das Variáveis Climáticas (modelo final):
   variavel  importancia
14      wc2     0.122864
13      wc2     0.110242
2       wc2     0.082226
11      wc2     0.074537
7       wc2     0.066583
16      wc2     0.056642
0       wc2     0.054611
17      wc2     0.050634
12      wc2     0.044062
6       wc2     0.042903
18      wc2     0.041502
1       wc2     0.037790
4       wc2     0.036793
15      wc2     0.036746
3       wc2     0.034740
9       wc2     0.033018
10      wc2     0.030724
5       wc2     0.021692
8       wc2     0.021691


### Previsão para cenários futuros

#### Definir a Função de Previsão

In [29]:
def prever_cenario(cenario_folder_path, modelo, output_path):
    """
    Função para prever a adequabilidade de habitat para um cenário climático futuro.
    """
    print(f"Processando cenário em: {cenario_folder_path}")
    
    # Listar e empilhar os rasters do cenário futuro
    cenario_files = sorted(glob.glob(os.path.join(cenario_folder_path, "*.tif")))
    
    # Ler o primeiro arquivo para obter o perfil (metadados)
    with rasterio.open(cenario_files[0]) as src:
        profile = src.profile
        
    # Ler todos os dados como um array numpy 3D (bandas, altura, largura)
    raster_data = np.stack([rasterio.open(f).read(1) for f in cenario_files])
    
    # Reorganizar o array para (altura, largura, bandas) para o modelo
    height, width = raster_data.shape[1], raster_data.shape[2]
    raster_data_reshaped = raster_data.reshape((len(cenario_files), -1)).T
    
    # Tratar valores NoData (geralmente -9999) para não quebrar o modelo
    nodata_val = -9999.0
    raster_data_reshaped[raster_data_reshaped == nodata_val] = np.nan
    
    # Preencher NaNs com a média da coluna (variável)
    col_mean = np.nanmean(raster_data_reshaped, axis=0)
    inds = np.where(np.isnan(raster_data_reshaped))
    raster_data_reshaped[inds] = np.take(col_mean, inds[1])

    print("Realizando a previsão...")
    previsao = modelo.predict_proba(raster_data_reshaped)[:, 1]
    
    # Remodelar a previsão de volta para o formato de raster (altura, largura)
    previsao_mapa = previsao.reshape((height, width))
    
    # Salvar o mapa de previsão como um novo GeoTIFF
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(previsao_mapa.astype(rasterio.float32), 1)
        
    print(f"Mapa de adequabilidade salvo em: {output_path}")
    return output_path

#### Executar as Previsões para todos os Períodos 

In [30]:
# --- CÉLULA DE DIAGNÓSTICO (APENAS PARA VERIFICAR) ---

print("--- Diagnóstico das Pastas de Clima Futuro ---")
periodos_futuros = [
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100"
]
clima_futuro_base_path = BASE_DIR / 'data' / 'clima_futuro'
pasta_com_problema = None

for periodo in periodos_futuros:
    pasta_cenario = clima_futuro_base_path / periodo
    print(f"\nVerificando a pasta: {pasta_cenario}")
    if not pasta_cenario.exists():
        print(f"❌ ERRO: A pasta '{periodo}' não existe!")
        pasta_com_problema = periodo; continue
    tif_files = sorted(glob.glob(os.path.join(pasta_cenario, "*.tif")))
    if not tif_files:
        print(f"❌ ERRO: A pasta '{periodo}' está vazia ou não contém arquivos .tif!")
        pasta_com_problema = periodo
    else:
        print(f"✅ Encontrados {len(tif_files)} arquivos .tif na pasta '{periodo}'.")

if pasta_com_problema:
    print(f"\n--- SOLUÇÃO ---\nA pasta '{pasta_com_problema}' está com problemas. Apague-a e extraia o .zip correspondente novamente.")
else:
    print("\n✅ Todas as pastas de cenário futuro parecem estar OK! Você pode prosseguir para a célula de previsão.")

--- Diagnóstico das Pastas de Clima Futuro ---

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080' não existe!

Verificando a pasta: c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100
❌ ERRO: A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100' não existe!

--- SOLUÇÃO ---
A pasta 'wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100' está com problemas. Apague-a e extraia o .zip correspondente novamente.


In [31]:
# --- CÉLULA DE EXECUÇÃO DAS PREVISÕES (APENAS PARA RODAR) ---

# Lista dos períodos futuros com os nomes EXATOS das suas pastas
periodos_futuros = [
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080",
    "wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100"
]

# Loop para prever e salvar cada cenário
for periodo in periodos_futuros:
    print(f"\n--- Iniciando previsão para o período: {periodo} ---")
    
    # O caminho para a pasta do cenário é construído dinamicamente
    cenario_path = BASE_DIR / 'data' / 'clima_futuro' / periodo
    
    # Verificação de segurança para garantir que a pasta e os arquivos existem
    if not cenario_path.exists():
        print(f"❌ ERRO: A pasta do cenário não foi encontrada em '{cenario_path}'. Pulando este período.")
        continue
    
    if not glob.glob(os.path.join(cenario_path, "*.tif")):
        print(f"❌ ERRO: Nenhum arquivo .tif encontrado em '{cenario_path}'. Pulando este período.")
        continue

    # O nome do arquivo de saída também usa o nome do período
    output_filename = f"previsao_trigona_{periodo}.tif"
    output_path = PASTA_PREVISOES / output_filename
    
    # Chamar a função de previsão
    prever_cenario(cenario_path, rf_model, output_path)

print("\n🎉 Todos os cenários futuros foram processados e salvos na pasta 'data/previsoes_futuras/'!")


--- Iniciando previsão para o período: wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040 ---
❌ ERRO: A pasta do cenário não foi encontrada em 'c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2021-2040'. Pulando este período.

--- Iniciando previsão para o período: wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060 ---
❌ ERRO: A pasta do cenário não foi encontrada em 'c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2041-2060'. Pulando este período.

--- Iniciando previsão para o período: wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080 ---
❌ ERRO: A pasta do cenário não foi encontrada em 'c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2061-2080'. Pulando este período.

--- Iniciando previsão para o período: wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_2081-2100 ---
❌ ERRO: A pasta do cenário não foi encontrada em 'c:\icev\extensao\abelhas_extensao\data\clima_futuro\wc2.1_10m_bioc_BCC-CSM2-MR_ssp245_20